In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import sys
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from multiprocessing import Pool
from tqdm.notebook import tqdm
import cv2
import scipy.stats as stats
from keras.models import load_model
from keras.utils import to_categorical, Sequence
from keras.backend import clear_session
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
from skimage.transform import AffineTransform, warp

import json
import math
import string
import collections

from six.moves import xrange
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import preprocess_input as _preprocess_input
import keras

import functools

In [ ]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

def upper(img,mergin=5):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=5):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=5):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=5):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size,tta=False,trim="edge"):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if tta:
        img = affine_image(img)
    if trim=="edge":
        img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img


def affine_image(img):
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)


In [ ]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(224,224),tta=False,trim="edge"):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.tta=tta
        self.trim=trim

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, i):
        _imgs=self.X[i * self.batch_size:(i + 1) * self.batch_size,:,:]
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,tta=self.tta,trim=self.trim))
        
        
        ret_y=[]
        for label,cat in zip(labels,[168,11,7]):
            ret_y.append(to_categorical(self.y[i * self.batch_size:(i + 1) * self.batch_size],num_classes=cat))
    
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y


In [ ]:
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None


def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils


def inject_keras_modules(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = keras.backend
        kwargs['layers'] = keras.layers
        kwargs['models'] = keras.models
        kwargs['utils'] = keras.utils
        return func(*args, **kwargs)

    return wrapper

backend = None
layers = None
models = None
keras_utils = None

def preprocess_input(x, **kwargs):
    kwargs = {k: v for k, v in kwargs.items() if k in ['backend', 'layers', 'models', 'utils']}
    return _preprocess_input(x, mode='torch', **kwargs)


def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """

        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return backend.tf.nn.swish(x)
            except AttributeError:
                pass

        return x * backend.sigmoid(x)
    return  swish


def get_dropout(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout
gm_exp = tf.Variable(3.0, dtype = tf.float32)

In [ ]:
%%time
tta=False
folderName="."

if tta:
    repeat=3
else:
    repeat=1

custom_objects= {'swish': inject_keras_modules(get_swish)(),'FixedDropout': inject_keras_modules(get_dropout)(),"tf":tf,"gm_exp":gm_exp}

for i in tqdm(range(4)):
    table = pq.read_table('./data/test_image_data_{0}.parquet'.format(i))
    dfEval = table.to_pandas()
    del table
    dfEval = dfEval.set_index("image_id")
    imgs = dfEval.values.reshape(-1,137,236)

    ans = pd.DataFrame(index=dfEval.index)
    del dfEval
    y_preds={"grapheme_root":[],"vowel_diacritic":[],"consonant_diacritic":[]}
    for file in tqdm(["multiEFFB0-justresize.h5"]):
        model = load_model("{0}/{1}".format(folderName, file),custom_objects =custom_objects, compile=False)
        #model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[2.,1.,1.])
        size = (model.input.shape[1],model.input.shape[2])
        print(file,size)
        for j in range(repeat):
            test_gen = DataLoader(imgs, [0]*imgs.shape[0], training=False, batch_size=128,size=size,tta=tta,trim=None)
            y_pred = model.predict(test_gen,verbose=1)
            #y_pred = np.argmax(y_pred, axis=1)
            for k,label in enumerate(["grapheme_root","vowel_diacritic","consonant_diacritic"]):
                tmp = y_preds[label]
                tmp.append(y_pred[k])
                y_preds[label]=tmp
        del model
        clear_session()
    for key in y_preds:
        y_pred = np.asarray(y_preds[key])
        #y_pred, _ = stats.mode(y_preds,axis=0)
        #y_pred = y_pred.reshape(-1,)
        y_pred = np.mean(y_preds[key],axis=0)
        y_pred = np.argmax(y_pred, axis=1)
        ans[key] = y_pred
    del y_preds
    with open("tmp{}.csv".format(i), "w") as fp:
        for row_id,consonant,grapheme,vowel in zip(ans.index, ans["consonant_diacritic"],ans["grapheme_root"],ans["vowel_diacritic"]):
            fp.write("{0}_grapheme_root,{1}\n".format(row_id,grapheme))
            fp.write("{0}_vowel_diacritic,{1}\n".format(row_id,vowel))
            fp.write("{0}_consonant_diacritic,{1}\n".format(row_id,consonant))